In [11]:
using Plots
using PLAYA
using Statistics
using UnicodeFun # to_latex
using DifferentialEquations
using DiffEqSensitivity
# using ODEInterfaceDiffEq # radau
using Catalyst

# Setup

In [12]:
# parse BOLOS output
bolos_interpolations = parse_bolos_output("../assets/air-kinetics-bolos.dat")

Loaded Mean energy from BOLOS file
Loaded Mobility *N from BOLOS file
Loaded Diffusion coefficient *N from BOLOS file
Loaded N2O->None from BOLOS file
Loaded NO->None from BOLOS file
Loaded NO->None from BOLOS file
Loaded NO->None from BOLOS file
Loaded O2(a1)->None from BOLOS file
Loaded O->None from BOLOS file
Loaded N2->None from BOLOS file
Loaded N->None from BOLOS file
Loaded O3->None from BOLOS file
Loaded O2->None from BOLOS file
Loaded N2O->N2O^- from BOLOS file
Loaded N2O->N2O^+ from BOLOS file
Loaded N2O->N2O(V2)(0.073eV) from BOLOS file
Loaded N2O->N2O(V3)(0.159eV) from BOLOS file
Loaded N2O->N2O(V1)(0.276eV) from BOLOS file
Loaded N2O->N2O(e1)(4.05eV) from BOLOS file
Loaded N2O->N2O(e2)(8.5eV) from BOLOS file
Loaded N2O->N2O(e3)(9.6eV) from BOLOS file
Loaded NO->O^-+N from BOLOS file
Loaded NO->NO(ionization) from BOLOS file
Loaded NO->NO^+ from BOLOS file
Loaded NO->N+O^+ from BOLOS file
Loaded NO->N^++O from BOLOS file
Loaded NO->NO++ from BOLOS file
Loaded NO->NO(a4PI) f

Dict{AbstractString,Interpolations.GriddedInterpolation{Float64,1,Float64,Interpolations.Gridded{Interpolations.Linear},Tuple{Array{Float64,1}}}} with 87 entries:
  "NO->NO(A2SIGMA+)"      => 200-element interpolate((::Array{Float64,1},), ::A…
  "N2->N2(v3)"            => 200-element interpolate((::Array{Float64,1},), ::A…
  "NO->NO(D2SIGMA+)"      => 200-element interpolate((::Array{Float64,1},), ::A…
  "N->N^+"                => 200-element interpolate((::Array{Float64,1},), ::A…
  "O2(a1)->None"          => 200-element interpolate((::Array{Float64,1},), ::A…
  "N2->N2(v6)"            => 200-element interpolate((::Array{Float64,1},), ::A…
  "N2->N2(v4)"            => 200-element interpolate((::Array{Float64,1},), ::A…
  "O2->O2(v1res)"         => 200-element interpolate((::Array{Float64,1},), ::A…
  "O2(a1)->O2^+"          => 200-element interpolate((::Array{Float64,1},), ::A…
  "O2->O^-+O"             => 200-element interpolate((::Array{Float64,1},), ::A…
  "O->O(1D)"              =

In [15]:
EN = 200. # Townsend
Tgas = 300.
Te = bolos_interpolations["Mean energy"](EN) * 1.1604505e4  # Temperature of electrons in K
dTion  = 2.0e0 / ( 3.0e0 * 1.3807e-16 ) * 1.6605e-24 * ( 1.0e-17 * EN )^2
TionN  = Tgas + dTion * 14.0e0 * 8.0e19^2
TionN2 = Tgas + dTion * 28.0e0 * 4.1e19^2
TionN3 = Tgas + dTion * 42.0e0 * 6.1e19^2
TionN4 = Tgas + dTion * 56.0e0 * 7.0e19^2
TeffN  = ( TionN  + 0.5e0 * Tgas ) / ( 1.0e0 + 0.5e0 )
TeffN2 = ( TionN2 + 1.0e0 * Tgas ) / ( 1.0e0 + 1.0e0 )
TeffN3 = ( TionN3 + 1.5e0 * Tgas ) / ( 1.0e0 + 1.5e0 )
TeffN4 = ( TionN4 + 2.0e0 * Tgas ) / ( 1.0e0 + 2.0e0 )
Teff3Q = 46.501347e-27*((1.87e-4*(1.0e5/(273.0*1.38064e-23))*( 1.0e-21 * EN ))^2)/(3.0e0*1.38064e-23) + Tgas

globals = Dict{Symbol, Any}([
                :E => EN,
                :Te => Te,
                :Tgas => Tgas,
                :dTion   => dTion,
                :TionN   => TionN,
                :TionN2  => TionN2,
                :TionN3  => TionN3,
                :TionN4  => TionN4,
                :TeffN   => TeffN,
                :TeffN2  => TeffN2,
                :TeffN3  => TeffN3,
                :TeffN4  => TeffN4,
                :Teff3Q  => Teff3Q,
               ])

# create functions that will get evaluated to rates
globals[:BOLOS] = (id, env) -> bolos_interpolations[id](env[:E])

println(globals)

Dict{Symbol,Any}(:TionN2 => 1809.503179546607,:dTion => 3.2070688781053097e-38,:TeffN3 => 2304.828553632216,:Te => 63815.573943526564,:TeffN4 => 3233.39900050699,:E => 200.0,:TionN4 => 9100.19700152097,:TeffN => 2215.689143188238,:Teff3Q => 1405.4034963438025,:TionN3 => 5312.07138408054,:BOLOS => var"#9#10"(),:TionN => 3173.5337147823575,:TeffN2 => 1054.7515897733035,:Tgas => 300.0)


In [16]:
# replace stuff so it is Catalyst-compliant
replacements = [
                "^+" => to_latex("^+"),
                "^-" => to_latex("^-"),
                "(" => to_latex("\\llcorner"),
                ")" => to_latex("\\lrcorner"),
                "4.5eV" => to_latex("4°5eV"),
                "`" => to_latex("\\tilde"),
                "=>" => "-->",
    ]

7-element Array{Pair{String,String},1}:
    "^+" => "⁺"
    "^-" => "⁻"
     "(" => "⌞"
     ")" => "⌟"
 "4.5eV" => "4°5eV"
     "`" => "̃"
    "=>" => "-->"

In [18]:
# parse the reaction file
(rs, ps) = parse_reactions("../assets/air-kinetics.reactions", globals; to_replace=replacements)

4.939058174439241e-17 *1e6
1.9911141532416576e-16 *1e6
1.797743750895955e-16 *1e6
7.067316923201697e-16 *1e6
5.767108659083016e-16 *1e6
1.784256590959922e-16 *1e6
1.424435933689807e-16 *1e6
3.819969134318789e-16 *1e6
1.8163166126193177e-16 *1e6
1.8730503987046348e-16 *1e6
4.057625093885372e-16 *1e6
7.504453417455757e-16 *1e6
1.7646814595265084e-16 *1e6
4.981545523119259e-16 *1e6
1.1505931286663538e-15 *1e6
3.972835714775334e-15 *1e6
1.3639781504135325e-17 *1e6
1.1505931286663538e-15 *1e6
2.366727790024083e-15 *1e6
1.6630317338546856e-16 *1e6
1.1096030276599076e-16 *1e6
2.8493172592219414e-16 *1e6
9.709953961429102e-17 *1e6
7.070913161706952e-16 *1e6
1.951697397109282e-16 *1e6
6.910712391498831e-16 *1e6
5.351699934483668e-16 *1e6
4.80874806454691e-18 *1e6
9.111030100148469e-19  *1e6
2.217650170397427e-19  *1e6
3.442467740972708e-16 *1e6
1.8432516749487616e-14 *1e6
4.1379529557079665e-17 *1e6
2.766759847456786e-17 *1e6
3.1316031291510663e-16 *1e6
1.3916818395449048e-16 *1e6
1.72739962435

(ReactionSystem(Reaction[Reaction{Variable,Int64}(p1, Operation[e(t), N2(t)], Operation[e(t), N2⌞A3⌟(t)], [1, 1], [1, 1], Pair{Variable,Int64}[N2 => -1, N2⌞A3⌟ => 1], false), Reaction{Variable,Int64}(p2, Operation[e(t), N2(t)], Operation[e(t), N2⌞A3⌟(t)], [1, 1], [1, 1], Pair{Variable,Int64}[N2 => -1, N2⌞A3⌟ => 1], false), Reaction{Variable,Int64}(p3, Operation[e(t), N2(t)], Operation[e(t), N2⌞A3⌟(t)], [1, 1], [1, 1], Pair{Variable,Int64}[N2 => -1, N2⌞A3⌟ => 1], false), Reaction{Variable,Int64}(p4, Operation[e(t), N2(t)], Operation[e(t), N2⌞B3⌟(t)], [1, 1], [1, 1], Pair{Variable,Int64}[N2⌞B3⌟ => 1, N2 => -1], false), Reaction{Variable,Int64}(p5, Operation[e(t), N2(t)], Operation[e(t), N2⌞B3⌟(t)], [1, 1], [1, 1], Pair{Variable,Int64}[N2⌞B3⌟ => 1, N2 => -1], false), Reaction{Variable,Int64}(p6, Operation[e(t), N2(t)], Operation[e(t), N2⌞B3⌟(t)], [1, 1], [1, 1], Pair{Variable,Int64}[N2⌞B3⌟ => 1, N2 => -1], false), Reaction{Variable,Int64}(p7, Operation[e(t), N2(t)], Operation[e(t), N2⌞ã1⌟

# Create the system of ODEs

In [19]:
# initial conditions
n_gas = 2.5e19
conc = Dict(["e" => 1e12, "N2" => 0.8*n_gas, "O2" => 0.2*n_gas])
u0 = [ get(conc, string(s), 0.) for s in species(rs) ]
tspan = (0., 1e-4)

odesys = convert(ODESystem, rs)

# Keep electrons and neutrals fixed
keep_fixed!(odesys, "e")
keep_fixed!(odesys, "ANY_NEUTRAL")

Converting to ODEs....

Equation(derivative(ANY_NEUTRAL(t), t), identity(0))

In [25]:
prob = ODEProblem(odesys, u0, tspan, ps)

print("Solving....")
sol = solve(prob, Rodas5(), reltol=1e-8, abstol=1e-8)

Solving....

retcode: Success
Interpolation: 3rd order Hermite
t: 2280-element Array{Float64,1}:
 0.0
 1.0202472135820603e-12
 1.4678470456232926e-12
 2.270590740173691e-12
 3.073334434724089e-12
 4.020558351303143e-12
 5.0468297221301e-12
 6.19749912885845e-12
 7.458432628507414e-12
 8.846862197554904e-12
 1.036314087308487e-11
 1.201648737322884e-11
 1.3811439977001117e-11
 ⋮
 9.207504689210106e-5
 9.275562675260839e-5
 9.344494334985538e-5
 9.41432419119388e-5
 9.485077836996342e-5
 9.556781995509043e-5
 9.629464587856357e-5
 9.70315480517254e-5
 9.777883199947168e-5
 9.853681737236077e-5
 9.930583936468431e-5
 0.0001
u: 2280-element Array{Array{Float64,1},1}:
 [1.0e12, 2.0e19, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0e18, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
 [1.0e12, 1.9999999932870177e19, 8.748024654621065e9, 2.9796871918417152e10, 1.4406820787240042e10, 3.818562570539634e9, 8.279562952290562e9, 8.27945003206327e9, 4.999999965289289e18, 3.828254984199735e9  …  28471.691854958153,

In [ ]:
plot(sol, vars=(0, 22), yscale=:log10, ylims=[1,3e12])
plot!(sol, vars=(0, 6))

# Sensitivity analysis

Based on [DifferentialEquations tutorial](https://diffeq.sciml.ai/stable/analysis/global_sensitivity/)

In [ ]:
f1 = function (p)
  prob1 = remake(prob;p=p)
  sol = solve(prob1,Rodas5();
              saveat=range(tspan[2]/10., stop=tspan[2], length=100),
              reltol=1e-8,
              abstol=1e-8,
             )
  mean(sol[6,:])/mean(sol[22,:])
end

bounds = [ [0.5*i, 1.5*i] for i in ps ]
m = gsa(f1, Morris(total_num_trajectory=1000, num_trajectory=150), bounds)

# output most significant reactions
inds = sortperm(m.means[1,:])
for i in reverse(inds)[1:5]
    r =  rs.eqs[i]
    f = x-> join(map(string, x), " + ")
    println(f(r.substrates), " => ", f(r.products), ":  μ = $(m.means[i]),  σ = $(m.variances[i])")
end